In [ ]:
from tbh import runner_tools as rt 
from tbh import plotting as pl

import multiprocessing as mp

mp.set_start_method("spawn")  # previously "forkserver"

studies_dict = rt.DEFAULT_STUDIES_DICT
params = rt.get_full_default_params(studies_dict)

# Metropolis config
n_cores = 1 # Requesting multiple cores won't work on a mac
tune = 50
draws = 205

# Full runs config
burn_in = int(draws / 2.) # 10000
full_runs_samples = 100

import pytensor
pytensor.config.cxx = '/usr/bin/clang++'



bcm = rt.get_bcm_object(rt.DEFAULT_MODEL_CONFIG, studies_dict, params)
idata = rt.run_metropolis_calibration(bcm, draws=draws, tune=tune, cores=n_cores)
import arviz as az

rhats = az.rhat(idata)
burnt_rhats = az.rhat(idata.sel(draw=range(burn_in, idata.sample_stats.sizes['draw'])))
print(f"Max R_hat for full chains: {rhats.to_array().max().item()}")
print(f"Max R_hat for burnt chains: {burnt_rhats.to_array().max().item()}")

full_runs, unc_df = rt.run_full_runs(bcm, idata, burn_in, full_runs_samples)

from pathlib import Path 
output_folder_path = Path.cwd() / "test_outputs"

pl.plot_traces(idata, burn_in, output_folder_path)

pl.plot_post_prior_comparison(idata, burn_in, list(bcm.priors.keys()), list(bcm.priors.values()), n_col=4, output_folder_path=output_folder_path)

import matplotlib.pyplot as plt

selected_outputs = bcm.targets.keys()

for output in selected_outputs:
    fig, ax = plt.subplots()
    pl.plot_model_fit_with_uncertainty(ax, unc_df, output, bcm, x_min=2010)

    if output_folder_path:
        plt.savefig(output_folder_path / f"{output}.jpg", facecolor="white", bbox_inches='tight')
        plt.close()